In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder as ohe
from sklearn.preprocessing import StandardScaler

In [ ]:
past = pd.read_csv('PastLoans.csv')
new = pd.read_csv('NewApplications_1_Round1.csv')
past.set_index(['id'], inplace=True)
new.set_index(['id'], inplace=True)

In [ ]:
print(past.shape)
print(past.head())
print(new.shape)
print(new.head())

In [ ]:
past.value_counts(['default'])

In [ ]:
past.groupby(by=['default']).mean()

In [ ]:
past1 = past.drop(['digital2', 'digital3'], axis=1)
past1.rename(columns={'digital1':'digital'}, inplace=True)
y1 = list(past1['default'])
X1 = past1.drop(['default'], axis=1)

past2 = past.drop(['digital1', 'digital3'], axis=1)
past2.rename(columns={'digital2':'digital'}, inplace=True)
y2 = list(past2['default'])
X2 = past2.drop(['default'], axis=1)

past3 = past.drop(['digital1', 'digital2'], axis=1)
past3.rename(columns={'digital3':'digital'}, inplace=True)
y3 = list(past3['default'])
X3 = past3.drop(['default'], axis=1)


In [ ]:
cat_features = ['sex', 'employment']
pass_features = ['married']
numerical_features = ['income']
numerical_pass_features = ['digital']

cat_encoder = ohe()
numerical_encoder = StandardScaler()

In [ ]:
pd.get_dummies(X1[cat_features], drop_first=True)

In [ ]:
X1[numerical_features] = numerical_encoder.fit_transform(X1[numerical_features])
X2[numerical_features] = numerical_encoder.fit_transform(X2[numerical_features])
X3[numerical_features] = numerical_encoder.fit_transform(X3[numerical_features])

dums = pd.get_dummies(X1[cat_features], drop_first=True)
X1.drop(cat_features, axis=1, inplace=True)
X2.drop(cat_features, axis=1, inplace=True)
X3.drop(cat_features, axis=1, inplace=True)

X1[dums.columns] = dums
X2[dums.columns] = dums
X3[dums.columns] = dums

In [ ]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2, random_state=42)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.2, random_state=42)
X_train3, X_test3, y_train3, y_test3 = train_test_split(X3, y3, test_size=0.2, random_state=42)

In [ ]:
# cat1 = ohe().fit(X1[cat_features])
# cat2 = ohe().fit(X2[cat_features])
# cat3 = ohe().fit(X3[cat_features])

In [ ]:
clf1 = LogisticRegression(random_state=42).fit(X_train1, y_train1)
clf2 = LogisticRegression(random_state=42).fit(X_train2, y_train2)
clf3 = LogisticRegression(random_state=42).fit(X_train3, y_train3)

In [ ]:
clf1.score(X_train1, y_train1)

In [ ]:
clf1.predict(X_test1).sum()